In [1]:
#Importacion de Librerias
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive

In [2]:
#Montamos la unidad de Drive al Workspace
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Estableciendo fecha de inicio y fin para el WC de noticias
url_base = 'https://elcomercio.pe/archivo/todas/'
fecha_inicio = '2023-08-25'
fecha_fin = '2023-10-05'

In [5]:
# Almacenamiento de fechas
dtdf = []

In [6]:
# Bucle para la recoleccion de noticias
current_fecha = fecha_inicio
while current_fecha <= fecha_fin:
    # URL Formateada (Fecha Actual)
    url = f'{url_base}{current_fecha}/'

    # Usamos REQUESTS para la URL
    response = requests.get(url)

    # Validacion del estado de respuesta
    if response.status_code == 200:
        content = response.text

        # Parsing HTML con BS4
        soup = BeautifulSoup(content, 'html.parser')

        # Elementos del HTML con informacion relevante
        fecha_elements = soup.find_all('span', class_='story-item__date-time')
        hora_elements = soup.find_all('span', class_='story-item__date-time')
        titulo_elements = soup.find_all('h2', class_='story-item__content-title')
        autor_elements = soup.find_all('a', class_='story-item__author block uppercase mt-10 font-thin text-xs text-gray-200')
        tema_elements = soup.find_all('a', class_='story-item__section story-item__section--desktop text-sm text-black md:mb-15 hidden')
        resumen_elements = soup.find_all('p', class_='story-item__subtitle')

        # Extraer el texto de las clases del HTML
        fechas = [fecha.text for fecha in fecha_elements[::2]]
        horas = [hora.text for hora in hora_elements[1::2]]
        titulos = [titulo.find('a').text for titulo in titulo_elements if titulo.find('a')]
        autores = [autor.text for autor in autor_elements]
        temas = [tema.text for tema in tema_elements]
        resumenes = [resumen.text for resumen in resumen_elements]

        # Verificar Longitud de listas
        min_length = min(len(fechas), len(horas), len(titulos), len(autores))

        fechas = fechas[:min_length]
        horas = horas[:min_length]
        titulos = titulos[:min_length]
        autores = autores[:min_length]

        # Creacion de un DF
        df = pd.DataFrame({'date': fechas, 'time': horas, 'title': titulos, 'author': autores, 'sbuject': temas, 'resume': resumenes})

        # Agregar Df's al array
        dtdf.append(df)
    else:
        print(f"Error: Unable to Acces {current_fecha}.")

    # Avanzar a la siguiente fecha
    current_fecha = pd.to_datetime(current_fecha) + pd.DateOffset(days=1)
    current_fecha = current_fecha.strftime('%Y-%m-%d')

# Concatenar todos los DataFrames en uno solo
output = pd.concat(dtdf, ignore_index=True)

# Especifica la ruta completa del archivo CSV en tu Google Drive
csv_filepath = '/content/drive/MyDrive/InteligenciaArtificial_II/WebScrape_Portocarrero/wc_elcomercio_output_text.csv'

# Guardar el DataFrame en un archivo CSV en Google Drive
output.to_csv(csv_filepath, index=False, escapechar='\\')

print(f"Se ha extraído y guardado el texto en '{csv_filepath}'.")
print(f"Total de Noticias Adquiridas: ",len(output))

Se ha extraído y guardado el texto en '/content/drive/MyDrive/InteligenciaArtificial_II/WebScrape_Portocarrero/wc_elcomercio_output_text.csv'.
Total de Noticias Adquiridas:  4200
